In [ ]:
#@title download mujoco

!wget -q https://www.roboti.us/download/mujoco200_linux.zip
!unzip -q mujoco200_linux.zip

#### Copy over `mjkey.txt` into `.../mujoco` before the following step

In [ ]:
MJC_PATH = "root/Deep_RL_course"
MJC_PATH

In [ ]:
#@title clone and install mujoco-py

%cd $MJC_PATH
!git clone https://github.com/openai/mujoco-py.git
%cd mujoco-py
!pip install -e.

#### Specify the key file path in .bashrc before the following step

In [ ]:
!pip --no-cache-dir install -r requirements.txt

In [ ]:
%cd mujoco-py
!pip install -r requirements.dev.txt
!pip install mujoco-py==2.0.2.2

In [1]:
## cythonize at the first import
import mujoco_py

In [ ]:
#@title clone homework repo

%cd $SYM_PATH
!git clone https://github.com/berkeleydeeprlcourse/homework_fall2020.git

In [2]:
#@title set up virtual display

from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
#@title test virtual display

#@markdown If you see a video of a four-legged ant fumbling about, setup is complete!

import gym
from cs285.infrastructure.colab_utils import (
    wrap_env,
    show_video
) 

env = wrap_env(gym.make("Ant-v2"))

observation = env.reset()
for i in range(100):
    env.render(mode='rgb_array')
    obs, rew, term, _ = env.step(env.action_space.sample() ) 
    if term:
      break;
            
env.close()
print('Loading video...')
show_video()

## Run Behavior Cloning (Problem 1)

In [4]:
#@title imports

import os
import time
import numpy as np

from cs285.infrastructure.rl_trainer import RL_Trainer
from cs285.agents.bc_agent import BCAgent
from cs285.policies.loaded_gaussian_policy import LoadedGaussianPolicy

%load_ext autoreload
%autoreload 2

In [5]:
import ffmpeg

def vidwrite(fn, images, framerate=10, vcodec='libx264'):
    if not isinstance(images, np.ndarray):
        images = np.asarray(images)
    n,channel,height,width = images.shape
    process = (
        ffmpeg
            .input('pipe:', format='rawvideo', pix_fmt='rgb24', s='{}x{}'.format(width, height))
            .output(fn, pix_fmt='yuv420p', vcodec=vcodec, r=framerate)
            .overwrite_output()
            .run_async(pipe_stdin=True)
    )
    for frame in images:
        process.stdin.write(
            frame
                .astype(np.uint8)
                .tobytes()
        )
    process.stdin.close()
    process.wait()

In [ ]:
!python /root/Deep_RL_course/homework/hw1/run_hw1.py \
--expert_policy_file cs285/policies/experts/Ant.pkl \
--env_name Ant-v2 --exp_name dagger_ant --n_iter 20 \
--do_dagger --expert_data cs285/expert_data/expert_data_Ant-v2.pkl \
--video_log_freq 2

In [ ]:
import numpy as np

%cd "/root/Deep_RL_course/homework/hw1/data/q2_dagger_ant_Ant-v2_03-12-2020_19-49-59"

array = np.load("train_rollouts_8.npy")
print(array.shape)
vidwrite("output.avi", array[0], framerate=20)

## Running DAgger (Problem 2)
Modify the settings above:
1. check the `do_dagger` box
2. set `n_iters` to `10`
and then rerun the code.

In [ ]:
!python /root/Deep_RL_course/homework/hw1/run_hw1.py \
--expert_policy_file cs285/policies/experts/Humanoid.pkl \
--env_name Humanoid-v2 --exp_name DAgger_human --n_iter 10 --do_dagger \
--expert_data cs285/expert_data/expert_data_Humanoid-v2.pkl \
--train_batch_size 1000 --eval_batch_size 500 --n_layers 4 \
--video_log_freq 1

In [8]:
import numpy as np

%cd "/root/Deep_RL_course/homework/hw1/data/q2_DAgger_human_Humanoid-v2_03-12-2020_19-58-59"

array = np.load("eval_rollouts_4.npy")
print(array.shape)
vidwrite("output.avi", array[0], framerate=67)

/root/Deep_RL_course/homework/hw1/data/q2_DAgger_human_Humanoid-v2_03-12-2020_19-58-59
(2, 22, 3, 500, 500)
